In [2]:
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
ssl.create_default_context=ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
intents = [
    {
        'tag': 'greeting',
        'patterns': ['how are you?', 'hey', 'hi', 'hello', 'whats up', 'yo', 'good morning', 'good evening'],
        'response': ['Good and what about you?', 'Hi there!', 'Hello!', 'Nothing much, you?', 'Hey, how’s it going?']
    },
    {
        'tag': 'goodbye',
        'patterns': ['bye', 'see you later', 'goodbye', 'take care', 'farewell', 'catch you later'],
        'response': ['Goodbye!', 'See you later!', 'Take care!', 'Bye, have a great day!', 'Catch you next time!']
    },
    {
        'tag': 'thanks',
        'patterns': ['thank you', 'thanks', 'thanks a lot', 'appreciate it', 'much obliged'],
        'response': ['You\'re welcome!', 'No problem!', 'Glad to help!', 'Anytime!', 'My pleasure!']
    },
    {
        'tag': 'weather',
        'patterns': ['what is the weather like?', 'how is the weather?', 'is it raining today?', 'will it snow?', 'do I need an umbrella?'],
        'response': ['The weather is great!', 'Looks like a sunny day!', 'It might rain, take an umbrella.', 'It’s a bit cloudy today.', 'Chilly and breezy outside!']
    },
    {
        'tag': 'name',
        'patterns': ['what is your name?', 'who are you?', 'can you tell me your name?', 'do you have a name?'],
        'response': ['I am your friendly chatbot!', 'You can call me ChatBot!', 'I am here to assist you!', 'I go by many names, but you can call me ChatBuddy!']
    },
    {
        'tag': 'age',
        'patterns': ['how old are you?', 'what is your age?', 'when were you created?', 'are you old?'],
        'response': ['I am timeless!', 'Age is just a number, and I don’t have one!', 'I was created to assist you!', 'Let’s just say, I never age!']
    },
    {
        'tag': 'joke',
        'patterns': ['tell me a joke', 'make me laugh', 'say something funny', 'crack a joke'],
        'response': [
            'Why don’t skeletons fight each other? Because they don’t have the guts!',
            'I told my wife she was drawing her eyebrows too high. She looked surprised!',
            'What do you call fake spaghetti? An impasta!',
            'Why did the scarecrow win an award? Because he was outstanding in his field!',
            'Why don’t scientists trust atoms? Because they make up everything!'
        ]
    },
    {
        'tag': 'food',
        'patterns': ['what should I eat?', 'suggest me some food', 'I am hungry', 'give me food ideas'],
        'response': ['How about a pizza?', 'Try some pasta!', 'A burger sounds good!', 'Maybe a healthy salad?', 'Sushi is always a great choice!']
    },
    {
        'tag': 'hobby',
        'patterns': ['what are your hobbies?', 'do you have a hobby?', 'what do you do for fun?'],
        'response': ['I like chatting with people like you!', 'Learning new things is my hobby!', 'I enjoy processing data and answering questions!', 'Helping people is my favorite hobby!']
    },
    {
        'tag': 'mood',
        'patterns': ['how are you feeling?', 'are you happy?', 'are you sad?', 'what’s your mood?'],
        'response': ['I am always here to help!', 'Feeling great, thanks for asking!', 'I don’t have feelings, but I’m here to chat!', 'I’m as happy as you are!']
    },
    {
        'tag': 'motivation',
        'patterns': ['motivate me', 'give me a motivational quote', 'I need inspiration'],
        'response': [
            'Believe in yourself and all that you are!',
            'The only limit to our realization of tomorrow is our doubts of today!',
            'You are capable of amazing things!',
            'Success is not final, failure is not fatal: it is the courage to continue that counts!'
        ]
    },
    {
        'tag': 'fact',
        'patterns': ['tell me a fact', 'give me a fun fact', 'I want to learn something new'],
        'response': [
            'Did you know honey never spoils?',
            'Octopuses have three hearts!',
            'Bananas are berries, but strawberries are not!',
            'A group of flamingos is called a "flamboyance"!',
            'Water can boil and freeze at the same time!'
        ]
    },
    {
        'tag': 'math',
        'patterns': ['solve 2+2', 'what is 10 times 5?', 'do math', 'can you calculate?'],
        'response': ['I can help with math! Try asking me a specific question.', '2+2 = 4!', '10 times 5 is 50!', 'Sure! What would you like me to calculate?']
    },
    {
        'tag': 'time',
        'patterns': ['what time is it?', 'can you tell me the time?', 'current time?'],
        'response': ['I don’t have a watch, but you can check your phone!', 'Time is relative, but I can fetch it for you!', 'Let me check... Oh wait, I don’t have a clock!']
    },
    {
        'tag': 'date',
        'patterns': ['what day is today?', 'tell me today’s date', 'which day is it?'],
        'response': ['Today is a great day!', 'I can check, but why not just look at your calendar?', 'Every day is a good day!']
    },
    {
        'tag': 'programming',
        'patterns': ['what is python?', 'do you know programming?', 'can you code?'],
        'response': ['Python is a powerful programming language!', 'Yes, I know a lot about programming!', 'I can write code, but I leave the hard work to you!']
    },
    {
        'tag': 'sports',
        'patterns': ['who won the last football match?', 'do you like sports?', 'tell me about sports'],
        'response': ['I don’t watch sports, but I can find the latest updates for you!', 'Football, basketball, cricket—so many great games!', 'Sports are exciting! Which one do you like?']
    },
    {
        'tag': 'music',
        'patterns': ['do you like music?', 'what is your favorite song?', 'recommend me a song'],
        'response': ['Music is amazing! What genre do you like?', 'I don’t have a favorite, but I love all kinds of music!', 'Try listening to some jazz, rock, or classical music!']
    },
    {
        'tag': 'movies',
        'patterns': ['suggest a movie', 'what is your favorite movie?', 'do you like movies?'],
        'response': ['There are so many great movies! What genre do you prefer?', 'I love movies with good storytelling!', 'How about a sci-fi movie or a comedy?']
    }
]
    